Install Package
<li>pip install <b>requests</b></li>
<li>pip install <b>pandas</b></li>
<li>pip install <b>sqlalchemy</b></li>
<li>pip install <b>pymysql</b></li>

In [2]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import pymysql as pymysql
from urllib.parse import quote
from pathlib import Path
import os

IndentationError: expected an indented block (connectionpool.py, line 1055)

Connect ZabbixAPI And Database

In [ ]:
ZABBIX_API_URL = "http://192.168.10.121/zabbix/api_jsonrpc.php"
UNAME = "Admin"
PWORD = "zabbix"
engine = create_engine('mysql+pymysql://softnix:%s@192.168.10.109/zabbix' % quote('softnix'))

เช็คไฟล์ CSV ดูว่ามีอยู่หรือเปล่า ถ้าไม่มีให้สร้าง Column มาให้

In [ ]:
pathFile = os.getcwd()+'/hosts.csv'
path_to_file = pathFile
path = Path(path_to_file)
if not path.is_file():
    with open('hosts.csv', 'w') as f:
        f.write('''hostname;visible;ip;type;port;hostgroup;template;community
Zabbix.Test1;behind1;172.19.23.101;Agent;10050;Hypervisors,Templates,xxx;Linux by Zabbix agent;''')

อ่านไฟล์ CSV

In [ ]:
readFile = pd.read_csv('./hosts.csv', sep=';')

ขอ Authen Key จาก Zabbix

In [ ]:
def Authen():
    r = requests.post(ZABBIX_API_URL,
                  json={
                      "jsonrpc": "2.0",
                      "method": "user.login",
                      "params": {
                          "username": UNAME,
                          "password": PWORD},
                      "id": 1
                  })
    AUTHTOKEN = r.json()["result"]
    return AUTHTOKEN

HostGroup -> HostID

In [ ]:
def hostgroup(host):
    strID = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "hostgroup.get",
        "params": {
            "output": "extend",
            "filter": {"name": [str(host)]}
        },
        "auth": Authen(),
        "id": 1
    })
    res = ''
    for i in strID.json()['result']:
        res = i['groupid']
    return res

Template Name -> Template ID

In [ ]:
def template(template):
    strID = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "template.get",
        "params": {
            "output": "extend",
            "filter": {"host": [str(template)]}
        },
        "auth": Authen(),
        "id": 1
    })
    res = ''
    for i in strID.json()['result']:
        res = i['templateid']
    return res

Hostname -> HostID

In [ ]:
def host(host):
    strID = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
            "method": "host.get",
            "params": {
                "filter": {"host": [str(host)]}
            },
            "auth": Authen(),
            "id": 1
    })
    res = ''
    for i in strID.json()['result']:
        res = i['hostid']
    return res

โดย Default ของ API จะให้เพิ่มไปเป็น SNMP Version 3 เท่านั้น เลยต้องมี Function มาเพิ่แปลงเป็น Version 2 <br>
<u><b style='color:red;'>** หากต้องการใช้ Version 3 สามารถข้ามส่วนนี้ไปได้</b></u>

In [ ]:
def edit_interface(hostid, community):
    try:
        sql = f"""
            UPDATE
                interface_snmp 
            SET
                version = '2',
                community = '{community}',
                securityname = '',
                contextname = ''
            WHERE
                interfaceid IN (
                SELECT
                    interfaceid 
                FROM
                    interface 
            WHERE
                hostid = '{hostid}')
        """
        engine.execute(sql)
        query = engine.execute(f"select host from hosts where hostid = {hostid}")
        print('Change Version SNMP to V2 Hostname = ', query.fetchone()[0])
    except Exception as e:
        print('Error to Change SNMP: ', e)

In [ ]:
def assign_group():
    group_list = pd.DataFrame()
    group_list['hostname'] = readFile['hostname']
    group_list['hostgroup'] = readFile['hostgroup'].str.split(',')
    group_explode = group_list.set_index(['hostname']).apply(pd.Series.explode).reset_index()
    return group_explode

In [ ]:
def mass_group():
    number = 0
    for index, row in assign_group().iterrows():
        number = number + 1
        group_id = hostgroup(row['hostgroup'])
        host_id = host(row['hostname'])
        r = requests.post(ZABBIX_API_URL, json={
            "jsonrpc": "2.0",
            "method": "hostgroup.massadd",
            "params": {
                "groups": [
                    {
                        "groupid": str(group_id)
                    }
                ],
                "hosts": [
                    {
                        "hostid": str(host_id)
                    }
                ]
            },
            "auth": Authen(),
            "id": 1
        })
        print(number, ': Add Hostgroup: [', row['hostname'],'] HostGroup [', row['hostgroup'],'] Successfully')
        

<b style='color: green;'>Loop File CSV เพิ่มบันทึกให้ครบทั้งหมดในไฟล์</b>

In [ ]:
def insert_host():
    for index, row in readFile.iterrows():
        index = index + 1
        group_id = hostgroup(row['hostgroup'].split(',')[0])
        templates_id = template(row['template'])
        try:
            if row['type'] == 'Agent':
                r = requests.post(
                        ZABBIX_API_URL,
                        json={
                            "jsonrpc": "2.0",
                            "method": "host.create",
                            "params": {
                                "host": str(row['hostname']),
                                "interfaces": [
                                    {
                                        "type": 1,
                                        "main": 1,
                                        "useip": 1,
                                        "ip": str(row['ip']),
                                        "dns": "",
                                        "port": str(row['port'])
                                    }
                                ],
                                "groups": [
                                    {
                                        "groupid": str(group_id)
                                    }
                                ],
                                "templates": [
                                    {
                                        "templateid": str(templates_id)
                                    }
                                ]
                            },
                            "auth": Authen(),
                            "id": 1
                        }
                )
                print(index,' Type: Agent ', ': Add Host: [', row['hostname'],'] Successfully')
            else:
                r = requests.post(
                        ZABBIX_API_URL,
                        json={
                            "jsonrpc": "2.0",
                            "method": "host.create",
                            "params": {
                                "host": str(row['hostname']),
                                "interfaces": [
                                    {
                                        "type": 2,
                                        "main": 1,
                                        "useip": 1,
                                        "ip": str(row['ip']),
                                        "dns": "",
                                        "port": str(row['port']),
                                        "details": {
                                            "version": 3,
                                            "bulk": 0,
                                            "securityname": "",
                                            "contextname": "",
                                            "securitylevel": 1
                                        }
                                    }
                                ],
                                "groups": [
                                    {
                                        "groupid": str(group_id)
                                    }
                                ],
                                "templates": [
                                    {
                                        "templateid": str(templates_id)
                                    }
                                ]
                            },
                            "auth": Authen(),
                            "id": 1
                        }
                )
                print(index,' Type: SNMP ', ': Add Host: [', row['hostname'],'] Successfully')
        except:
                print(index, ': Add Host: [', row['hostname'],'] Error!')
                continue

แก้ไข Interface เป็น SNMP V2 และเพิ่ม Community

In [ ]:
def interface_snmp():
    for index, row in readFile.iterrows():
        host_id = host(row['hostname'])
        if row['type'] == "SNMP":
            edit_interface(host_id, row['community'])

เพิ่ม Host Group หากมี Host Group มากกว่า 1

In [ ]:
if __name__ == '__main__':
    print('######### Insert Hostname and Interface #########')
    insert_host()
    print('######### Insert Multiple Host Group #########')
    mass_group()
    print('######### Change Interface SNMP to V2 #########')
    interface_snmp()
    print('######### ๐Task Running Completely๐ #########')
